In [143]:
import pandas as pd
import numpy as np
from collections import defaultdict
import random
import itertools

In [144]:

def generate_exact_matrix(rows, cols, p_ones):
    total_cells = rows * cols
    number_of_ones = int(total_cells * p_ones)
    flat = np.array([1] * number_of_ones + [0] * (total_cells - number_of_ones))
    np.random.shuffle(flat)  
    return flat.reshape((rows, cols))


In [145]:
def generate_rand_matrix(rows, cols, p_ones):
    total_cells = rows * cols
    number_of_ones = int(total_cells * p_ones)
    flat = np.array([1] * number_of_ones + [0] * (total_cells - number_of_ones))
    np.random.shuffle(flat)  
    return flat.reshape((rows, cols))

# def get_neighbors(mat):
#     neighbor_dict = defaultdict(list)
#     n,m =mat.shape[0],mat.shape[1]
#     for i in range(n):
#         for j in range(m):
#             x = [[i+y[0], j+y[1], 0] for y in itertools.product([1,0,-1], [1,0,-1])]
#             z = [e for e in x if (0 <= e[0] < n and 0 <= e[1] < m and e[:2] != [i, j])]
#             neighbor_dict[(i, j)] = z

#     for k,v in neighbor_dict.items():
#         for e in v:
#             if mat[e[0],e[1]]==1:
#                 e[2]=1
#             else:
#                 e[2]=0

#     return neighbor_dict


# def next_gen(mat):
#     neighbor_sum_dict = defaultdict(int)
#     mat_copy = mat.copy()
#     neighbors = get_neighbors(mat)

#     for k,v in neighbors.items():
#         neighbors_sum = sum([e[2] for e in v])
        
#         neighbor_sum_dict[k] = neighbors_sum
        
#     for k,v in neighbor_sum_dict.items():
#         i, j = k

#         if (v < 2 or v > 3) and mat[i, j] == 1:
#             mat_copy[i, j] = 0

#         elif mat[i, j] == 0 and v == 3:
#             mat_copy[i, j] = 1
            
#     return mat_copy



def indvidual_matrix_data_collection(mat,gen_num,run_num,collection_dict,alive_percent,matrix_size):

    collection_dict["gen_dict"]["alive_cells"].append(int(mat.sum()))
    collection_dict["gen_dict"]["gen"].append(gen_num)
    collection_dict["gen_dict"]["run"].append(run_num)
    collection_dict["gen_dict"]["initial_percent_alive"].append(alive_percent)
    collection_dict["gen_dict"]["matrix_size"].append(matrix_size)

    return collection_dict

     

In [146]:
def get_neighbors(mat):
    n, m = mat.shape
    neighbor_dict = defaultdict(list)
    
    for i in range(n):
        for j in range(m):
            for dx, dy in itertools.product([-1, 0, 1], [-1, 0, 1]):
                if dx == 0 and dy == 0:
                    continue
                ni, nj = i + dx, j + dy
                if 0 <= ni < n and 0 <= nj < m:
                    neighbor_dict[(i, j)].append(mat[ni, nj])
    
    return neighbor_dict

def next_gen(mat):
    # n, m = mat.shape
    mat_copy = mat.copy()
    neighbors = get_neighbors(mat)

    for (i, j), neighbor_vals in neighbors.items():
        alive_neighbors = sum(neighbor_vals)
        
        if mat[i, j] == 1:
            if alive_neighbors < 2 or alive_neighbors > 3:
                mat_copy[i, j] = 0
        else:
            if alive_neighbors == 3:
                mat_copy[i, j] = 1

    return mat_copy

In [87]:


def run_game(mat,run_num,matrix_size,master_data_collection_dict,alive_percent,max_steps=101):

    indvidual_matrix_data_collection(mat,0,run_num,master_data_collection_dict,alive_percent,matrix_size)

    step = 1
    steady_count = 0  

    while step < max_steps:
        new_mat = next_gen(mat)
        indvidual_matrix_data_collection(new_mat,step,run_num,master_data_collection_dict,alive_percent,matrix_size)

        if np.array_equal(new_mat, mat):
            steady_count += 1
        else:
            steady_count = 0 

        if steady_count == 2:
            master_data_collection_dict["run_dict"]["term_reason"].append("steady")
            master_data_collection_dict["run_dict"]["last_gen"].append(step)
            master_data_collection_dict["run_dict"]["run"].append(run_num)
            master_data_collection_dict["run_dict"]["initial_percent_alive"].append(alive_percent)
            master_data_collection_dict["run_dict"]["matrix_size"].append(matrix_size)
            break
        
        if np.all(new_mat == 0):
            master_data_collection_dict["run_dict"]["term_reason"].append("dead")
            master_data_collection_dict["run_dict"]["last_gen"].append(step)
            master_data_collection_dict["run_dict"]["run"].append(run_num)
            master_data_collection_dict["run_dict"]["initial_percent_alive"].append(alive_percent)
            master_data_collection_dict["run_dict"]["matrix_size"].append(matrix_size)
            break
        
        mat = new_mat  
        step += 1
        
    if step == max_steps:
        master_data_collection_dict["run_dict"]["term_reason"].append("max")
        master_data_collection_dict["run_dict"]["last_gen"].append(step)
        master_data_collection_dict["run_dict"]["run"].append(run_num)
        master_data_collection_dict["run_dict"]["initial_percent_alive"].append(alive_percent)
        master_data_collection_dict["run_dict"]["matrix_size"].append(matrix_size)
     

In [97]:
sample_result_dict = defaultdict(lambda: defaultdict(list))

In [98]:
for m in range(10,110,10):
    for i in np.arange(.05, 1, 0.05):
        p = round(i, 2)  
        for x in range(11):  
            matrix = generate_rand_matrix(m, m, p)
            run_game(matrix, x, m*m, sample_result_dict, p)

sample_output_dict = {k:pd.DataFrame(v) for k,v in sample_result_dict.items()}
sample_generation_df = sample_output_dict["gen_dict"]
sample_run_df = sample_output_dict["run_dict"]

In [100]:
sample_output_df = pd.merge(sample_generation_df,sample_run_df,on=["run","initial_percent_alive","matrix_size"],how="left")

In [114]:
generate_rand_matrix(10, 10, .05).sum()

np.int64(7)

In [105]:
sample_generation_df.shape

(153691, 5)

In [102]:
sample_output_df.to_csv(r"C\:\\Users\\Will\\Documents\\school_data\\gol_new.csv",index= False)

In [ ]:
sample_output_df.to_csv(r"C\:\\Users\\Will\\Documents\\school_data\\gol.csv",index= False)

In [59]:
result_dict = defaultdict(lambda: defaultdict(list))

In [58]:
sample_amount_dict = {100: 73,
 400: 45,
 900: 45,
 1600: 45,
 2500: 45,
 3600: 45,
 4900: 45,
 6400: 45,
 8100: 45,
 10000: 45}

In [60]:
for m in range(10,110,10):
    for i in np.arange(.05, 1, 0.05):
        p = round(i, 2)  
        for x in range(sample_amount_dict[m*m]):  
            matrix = generate_rand_matrix(m, m, p)
            run_game(matrix, x, m*m, result_dict, p)

output_dict = {k:pd.DataFrame(v) for k,v in result_dict.items()}
generation_df = output_dict["gen_dict"]
run_df = output_dict["run_dict"]

In [61]:
output_df = pd.merge(generation_df,run_df,on=["run","initial_percent_alive","matrix_size"],how="left")

In [63]:
output_df.to_csv(r"C\:\\Users\\Will\\Documents\\school_data\\gol_final.csv",index= False)